# Analysis of synonymous codon usage biases in *Arabidopsis thaliana*, in relation to gene expression

## 1. Get and process data

### 1.1 Download original datasets

The data sources are:
- for RnaSeq data: https://www.ebi.ac.uk/gxa/experiments-content/E-CURD-1/resources/ExperimentDownloadSupplier.RnaSeqBaseline/tpms.tsv
- for CDS sequences: https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_genes.201606.cds.fasta.gz

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 1000)

In [2]:
from data_utils import download_expression_data, download_cds_data, load_expression_data, load_cds_data

C:\Users\VandB14\AppData\Local\Continuum\anaconda3\lib\site-packages\Bio\__init__.py:128: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: C:\Users\VandB14\AppData\Local\Continuum\anaconda3\lib\site-packages.
  format(_parent_dir), BiopythonWarning)


In [3]:
expr_fn = download_expression_data()
cds_fn = download_cds_data()

expr_data = load_expression_data(expr_fn)
cds_data = load_cds_data(cds_fn)

data/raw/E-CURD-1-query-results.tpms.tsv
data/raw/Araport11_genes.201606.cds.fasta


### 1.2 Preprocess the data

- In the RnaSeq data we calculate for each gene the maximal TPM value across samples (tissues and developmental stages)
- In the CDS data, when there are multiple transcripts per gene, we only keep the longest CDS sequence per gene.

We merge the RnaSeq and CDS sequences in one data frame, joined by Gene ID

In [4]:
from data_utils import process_data

In [5]:
cds_expr = process_data(expr_data, cds_data)
print(cds_expr)

data/processed/cds_expr.txt
         Gene ID Gene Name  Max TPM Transcript ID  Length  \
0      AT1G01010    NAC001     46.0   AT1G01010.1    1290   
1      AT1G01020      ARV1     27.0   AT1G01020.1     738   
2      AT1G01030      NGA3     12.0   AT1G01030.1    1077   
3      AT1G01040      DCL1     25.0   AT1G01040.2    5733   
4      AT1G01050      PPA1    172.0   AT1G01050.1     639   
...          ...       ...      ...           ...     ...   
26860  ATMG01350   ORF145C     31.0   ATMG01350.1     438   
26861  ATMG01360      COX1    706.0   ATMG01360.1    1584   
26862  ATMG01370   ORF111D     37.0   ATMG01370.1     336   
26863  ATMG01400   ORF105B      8.0   ATMG01400.1     318   
26864  ATMG01410    ORF204     12.0   ATMG01410.1     615   

                                            CDS sequence  
0      ATGGAGGATCAAGTTGGGTTTGGGTTCCGTCCGAACGACGAGGAGC...  
1      ATGGCGGCGAGTGAACACAGATGCGTGGGATGTGGTTTTAGGGTAA...  
2      ATGGATCTATCCCTGGCTCCGACAACAACAACAAGTTCCGACCAAG...  
3  

### 1.3 Translate and encode sequences

For analysis of codon biases it may be useful to translate the CDS sequences into codon sequences and amino acid sequences, and also to one-hot encode them to allow vectorized operations with Numpy.

To transform the list of sequences in numpy arrays, we need to zero pad the one-hot encoded sequences so that their length is equal to the maximal length of all sequences.

In [6]:
from sequence_utils import __DNA_MAPPING__, __CODON_MAPPING__, __AMINO_ACID_MAPPING__, __CODON_DICT__, __TRL_MAT__
from sequence_utils import one_hot_encode, translate_one_hot, padding_one_hot

print('\n1-hot encode to dna sequences:')
cds_dna_1hot = [one_hot_encode(seq, __DNA_MAPPING__) for seq in tqdm(cds_expr['CDS sequence'])]

print('\napply padding and make numpy array of 1-hot encoded dna sequences:')
cds_dna_1hot = padding_one_hot(cds_dna_1hot)

print('\n1-hot encode to codon sequences:')
cds_cod_1hot = [one_hot_encode(seq, __CODON_MAPPING__) for seq in tqdm(cds_expr['CDS sequence'])]

print('\ntranslate into 1-hot encoded amino acid sequences:')
cds_aa_1hot = [translate_one_hot(seq_1hot) for seq_1hot in tqdm(cds_cod_1hot)]

print('\napply padding and make numpy array of 1-hot encoded codon sequences:')
cds_cod_1hot = padding_one_hot(cds_cod_1hot)

print('\napply padding and make numpy array of 1-hot encoded amino acid sequences:')
cds_aa_1hot = padding_one_hot(cds_aa_1hot)

100%|██████████| 26865/26865 [00:09<00:00, 2882.28it/s]



1-hot encode to dna sequences:

apply padding and make numpy array of 1-hot encoded dna sequences:

1-hot encode to codon sequences:

translate into 1-hot encoded amino acid sequences:

apply padding and make numpy array of 1-hot encoded codon sequences:

apply padding and make numpy array of 1-hot encoded amino acid sequences:


In [7]:
print(f"cds_dna_hot.shape: ({cds_dna_1hot.shape[0]}, {cds_dna_1hot.shape[1]}, {cds_dna_1hot.shape[2]})")
print(f"cds_cod_hot.shape: ({cds_cod_1hot.shape[0]}, {cds_cod_1hot.shape[1]}, {cds_cod_1hot.shape[2]})")
print(f"cds_aa_hot.shape: ({cds_aa_1hot.shape[0]}, {cds_aa_1hot.shape[1]}, {cds_aa_1hot.shape[2]})")

cds_dna_hot.shape: (26865, 16203, 4)
cds_cod_hot.shape: (26865, 5401, 64)
cds_aa_hot.shape: (26865, 5401, 21)


## 2. Definitions

Some of the following definitions may be useful for your analysis. It is not necessary to use all of these in your analysis, and you may also use other measures. Use them where you see value for interpreting the relationship between biases in synonymous codon usage and expression levels.

- $x_{ij}$: **number of occurences** of the $j$-th codon for the $i$-th amino acid
- $n_i$: **number of alternative codons** for the $i$-th amino acid
- $\sum_{j=1}^{n_i}x_{ij}$: **number of amino acid occurences** for the $i$-th amino acid (repeated for corresponding $j$-th codons)
- $\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}$: the $j$-th **expected codon frequency** under the assumption of equal usage of the synonymous codons for an $i$-th amino acid
- $\text{RSCU}_{ij}$: **relative synonymous codon usage**, the observed frequency for a codon divided by the frequency expected under the assumption of equal usage of the synonymous codons for an amino acid

\begin{equation}
\text{RSCU}_{ij} = \frac{x_{ij}}{\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}}
\end{equation}
- $\text{RSCU}_{i_\text{max}}$: **RSCU for the most frequently used codon** for the $i$-th amino acid
- $\text{x}_{i_\text{max}}$: **codon frequency for the most frequently used codon** for the $i$-th amino acid
- $w_{ij}$: **relative adaptiveness** of the $j$-th codon for the $i$-th amino acid

\begin{equation}
w_{ij} = \frac{\text{RSCU}_{ij}}{\text{RSCU}_{i_\text{max}}} = \frac{x_{ij}}{x_{i_{\text{max}}}}
\end{equation}
- $L$: **length of a gene** in number of codons
- $\text{CAI}$: **codon adaptation index (CAI)** for a CDS is the geometric mean of the RSCU values corresponding to each of the codons used in that CDS, divided by the maximum possible CAI for a CDS of the same amino acid composition

\begin{equation}
\text{CAI}=\frac{\text{CAI}_\text{obs}}{\text{CAI}_\text{max}} = \frac{\left(\prod_{k=1}^L \text{RSCU}_k\right)^\frac{1}{L}}{\left(\prod_{k=1}^L \text{RSCU}_{k_\text{max}}\right)^\frac{1}{L}}=\left(\prod_{k=1}^L w_k\right)^{\frac{1}{L}}
\end{equation}


## 3. Start of analysis

- Add markdown text to explain your analysis steps
- Use graphs and/or tables
- Posit biological interpretations or hypotheses

## 2. Definitions

Some of the following definitions may be useful for your analysis. It is not necessary to use all of these in your analysis, and you may also use other measures. Use them where you see value for interpreting the relationship between biases in synonymous codon usage and expression levels.

- $x_{ij}$: **number of occurences** of the $j$-th codon for the $i$-th amino acid
- $n_i$: **number of alternative codons** for the $i$-th amino acid
- $\sum_{j=1}^{n_i}x_{ij}$: **number of amino acid occurences** for the $i$-th amino acid (repeated for corresponding $j$-th codons)
- $\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}$: the $j$-th **expected codon frequency** under the assumption of equal usage of the synonymous codons for an $i$-th amino acid
- $\text{RSCU}_{ij}$: **relative synonymous codon usage**, the observed frequency for a codon divided by the frequency expected under the assumption of equal usage of the synonymous codons for an amino acid

\begin{equation}
\text{RSCU}_{ij} = \frac{x_{ij}}{\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}}
\end{equation}
- $\text{RSCU}_{i_\text{max}}$: **RSCU for the most frequently used codon** for the $i$-th amino acid
- $\text{x}_{i_\text{max}}$: **codon frequency for the most frequently used codon** for the $i$-th amino acid
- $w_{ij}$: **relative adaptiveness** of the $j$-th codon for the $i$-th amino acid

\begin{equation}
w_{ij} = \frac{\text{RSCU}_{ij}}{\text{RSCU}_{i_\text{max}}} = \frac{x_{ij}}{x_{i_{\text{max}}}}
\end{equation}
- $L$: **length of a gene** in number of codons
- $\text{CAI}$: **codon adaptation index (CAI)** for a CDS is the geometric mean of the RSCU values corresponding to each of the codons used in that CDS, divided by the maximum possible CAI for a CDS of the same amino acid composition

\begin{equation}
\text{CAI}=\frac{\text{CAI}_\text{obs}}{\text{CAI}_\text{max}} = \frac{\left(\prod_{k=1}^L \text{RSCU}_k\right)^\frac{1}{L}}{\left(\prod_{k=1}^L \text{RSCU}_{k_\text{max}}\right)^\frac{1}{L}}=\left(\prod_{k=1}^L w_k\right)^{\frac{1}{L}}
\end{equation}


## 3. Start of analysis

- Add markdown text to explain your analysis steps
- Use graphs and/or tables
- Posit biological interpretations or hypotheses

## 2. Definitions

Some of the following definitions may be useful for your analysis. It is not necessary to use all of these in your analysis, and you may also use other measures. Use them where you see value for interpreting the relationship between biases in synonymous codon usage and expression levels.

- $x_{ij}$: **number of occurences** of the $j$-th codon for the $i$-th amino acid
- $n_i$: **number of alternative codons** for the $i$-th amino acid
- $\sum_{j=1}^{n_i}x_{ij}$: **number of amino acid occurences** for the $i$-th amino acid (repeated for corresponding $j$-th codons)
- $\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}$: the $j$-th **expected codon frequency** under the assumption of equal usage of the synonymous codons for an $i$-th amino acid
- $\text{RSCU}_{ij}$: **relative synonymous codon usage**, the observed frequency for a codon divided by the frequency expected under the assumption of equal usage of the synonymous codons for an amino acid

\begin{equation}
\text{RSCU}_{ij} = \frac{x_{ij}}{\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}}
\end{equation}
- $\text{RSCU}_{i_\text{max}}$: **RSCU for the most frequently used codon** for the $i$-th amino acid
- $\text{x}_{i_\text{max}}$: **codon frequency for the most frequently used codon** for the $i$-th amino acid
- $w_{ij}$: **relative adaptiveness** of the $j$-th codon for the $i$-th amino acid

\begin{equation}
w_{ij} = \frac{\text{RSCU}_{ij}}{\text{RSCU}_{i_\text{max}}} = \frac{x_{ij}}{x_{i_{\text{max}}}}
\end{equation}
- $L$: **length of a gene** in number of codons
- $\text{CAI}$: **codon adaptation index (CAI)** for a CDS is the geometric mean of the RSCU values corresponding to each of the codons used in that CDS, divided by the maximum possible CAI for a CDS of the same amino acid composition

\begin{equation}
\text{CAI}=\frac{\text{CAI}_\text{obs}}{\text{CAI}_\text{max}} = \frac{\left(\prod_{k=1}^L \text{RSCU}_k\right)^\frac{1}{L}}{\left(\prod_{k=1}^L \text{RSCU}_{k_\text{max}}\right)^\frac{1}{L}}=\left(\prod_{k=1}^L w_k\right)^{\frac{1}{L}}
\end{equation}


## 3. Start of analysis

- Add markdown text to explain your analysis steps
- Use graphs and/or tables
- Posit biological interpretations or hypotheses

cds_dna_hot.shape: (26865, 16203, 4)
cds_cod_hot.shape: (26865, 5401, 64)
cds_aa_hot.shape: (26865, 5401, 21)


## 2. Definitions

Some of the following definitions may be useful for your analysis. It is not necessary to use all of these in your analysis, and you may also use other measures. Use them where you see value for interpreting the relationship between biases in synonymous codon usage and expression levels.

- $x_{ij}$: **number of occurences** of the $j$-th codon for the $i$-th amino acid
- $n_i$: **number of alternative codons** for the $i$-th amino acid
- $\sum_{j=1}^{n_i}x_{ij}$: **number of amino acid occurences** for the $i$-th amino acid (repeated for corresponding $j$-th codons)
- $\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}$: the $j$-th **expected codon frequency** under the assumption of equal usage of the synonymous codons for an $i$-th amino acid
- $\text{RSCU}_{ij}$: **relative synonymous codon usage**, the observed frequency for a codon divided by the frequency expected under the assumption of equal usage of the synonymous codons for an amino acid

\begin{equation}
\text{RSCU}_{ij} = \frac{x_{ij}}{\frac{1}{n_i}\sum_{j=1}^{n_i}x_{ij}}
\end{equation}
- $\text{RSCU}_{i_\text{max}}$: **RSCU for the most frequently used codon** for the $i$-th amino acid
- $\text{x}_{i_\text{max}}$: **codon frequency for the most frequently used codon** for the $i$-th amino acid
- $w_{ij}$: **relative adaptiveness** of the $j$-th codon for the $i$-th amino acid

\begin{equation}
w_{ij} = \frac{\text{RSCU}_{ij}}{\text{RSCU}_{i_\text{max}}} = \frac{x_{ij}}{x_{i_{\text{max}}}}
\end{equation}
- $L$: **length of a gene** in number of codons
- $\text{CAI}$: **codon adaptation index (CAI)** for a CDS is the geometric mean of the RSCU values corresponding to each of the codons used in that CDS, divided by the maximum possible CAI for a CDS of the same amino acid composition

\begin{equation}
\text{CAI}=\frac{\text{CAI}_\text{obs}}{\text{CAI}_\text{max}} = \frac{\left(\prod_{k=1}^L \text{RSCU}_k\right)^\frac{1}{L}}{\left(\prod_{k=1}^L \text{RSCU}_{k_\text{max}}\right)^\frac{1}{L}}=\left(\prod_{k=1}^L w_k\right)^{\frac{1}{L}}
\end{equation}


## 3. Start of analysis

- Add markdown text to explain your analysis steps
- Use graphs and/or tables
- Posit biological interpretations or hypotheses